# Planet API Python Client


This tutorial is an introduction to [Planet](https://www.planet.com)'s Data and Orders API using the official [Python client](https://github.com/planetlabs/planet-client-python), the `planet` module. It shows you how to create bulk orders, use our tools, and deliver to the cloud.

## Requirements

This tutorial assumes familiarity with the [Python](https://python.org) programming language throughout. Python modules used in this tutorial are:
* [IPython](https://ipython.org/) and [Jupyter](https://jupyter.org/)
* [planet](https://github.com/planetlabs/planet-client-python)
* [geojsonio](https://pypi.python.org/pypi/geojsonio)
* [rasterio](https://rasterio.readthedocs.io/en/latest/index.html)
* [asyncio](https://docs.python.org/3/library/asyncio.html)

You should also have an account on the Planet Platform and retrieve your API key from your [account page](https://www.planet.com/account/).

## Useful links 
* [Planet Client V2 Documentation](https://github.com/planetlabs/planet-client-python)
* [Planet Data API reference](https://developers.planet.com/docs/apis/data/)

This tutorial will cover the basic operations possible with the Python client, particularly those that interact with the Data API and Orders API

## Set up

In order to interact with the Planet API using the client, we need to import the necessary packages & define helper functions.

In [1]:
#general packages
import os
import json
import glob
import asyncio
import requests
import nest_asyncio 
import matplotlib.pyplot as plt
from requests.auth import HTTPBasicAuth
from datetime import datetime, timedelta

#geospatial packages
import rasterio
import geopandas as gpd
from shapely.geometry import shape
from shapely.ops import unary_union


#planet SDK
from planet import Auth, reporting, Session, OrdersClient, order_request, data_filter


# We will also create a small helper function to print out JSON with proper indentation.
def indent(data):
    print(json.dumps(data, indent=2))

We next need to create a `client` object registered with our API key. The API key will be automatically read from the `PL_API_KEY` environment variable if it exists. If not, you can provide it below. You can also authenticate via the CLI using [`auth init`](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/cli/cli-reference/?h=auth#auth:~:text=message%20and%20exit.-,auth,-%C2%B6), this will store your API key as an environment variable.

In [9]:

API_KEY = input("Paste API Key here")
os.environ['PL_API_KEY'] = API_KEY

client = Auth.from_key(API_KEY)

## Searching

We can search for items that are interesting by using the `quick_search` member function. Searches, however, always require a proper request that includes a filter that selects the specific items to return as seach results.

Let's also read in a GeoJSON geometry into a variable so we can use it during testing. The geometry can only have one polygon to work with the data API

In [10]:
with open("lake_shasta.geojson") as f:
    geom_all = json.loads(f.read())['features'][0]['geometry']
geom_large = {
    "type": "Polygon",
    "coordinates": [
        [[-122.48691050204125,40.7049384269217],
         [-122.11474863680688,40.7049384269217],
         [-122.11474863680688,40.898294348395034],
         [-122.48691050204125,40.898294348395034],
         [-122.48691050204125,40.7049384269217]]]
}
print(geom_all)

{'type': 'Polygon', 'coordinates': [[[-122.41199233711868, 40.80123709923839], [-122.33296384724517, 40.80119907248327], [-122.33318988290331, 40.83315984019974], [-122.41192371590644, 40.83344073039331], [-122.41199233711868, 40.80123709923839]]]}


### Filters

The possible filters include `and_filter`, `date_range_filter`, `range_filter` and so on, mirroring the options supported by the Planet API. Additional filters are described [here](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/python/sdk-guide/#filter:~:text=(main())-,Filter,-%C2%B6).

In [11]:
# Define the filters we'll use to find our data

item_types = ["PSScene"]

#Geometry filter
geom_filter = data_filter.geometry_filter(geom_large)

#Date range filter
date_range_filter = data_filter.date_range_filter(
    "acquired", gt = datetime(month=11, day=1, year=2022),
    lt = datetime(month=3, day=1, year=2023))
#Cloud cover filter
#cloud_cover_filter = data_filter.range_filter('clear_percent', gt = 80)

#Combine all of the filters
combined_filter = data_filter.and_filter([geom_filter, date_range_filter])#, cloud_cover_filter])

In [12]:
combined_filter

{'type': 'AndFilter',
 'config': [{'type': 'GeometryFilter',
   'field_name': 'geometry',
   'config': {'type': 'Polygon',
    'coordinates': [[[-122.48691050204125, 40.7049384269217],
      [-122.11474863680688, 40.7049384269217],
      [-122.11474863680688, 40.898294348395034],
      [-122.48691050204125, 40.898294348395034],
      [-122.48691050204125, 40.7049384269217]]]}},
  {'type': 'DateRangeFilter',
   'field_name': 'acquired',
   'config': {'gt': '2022-11-01T00:00:00Z', 'lt': '2023-03-01T00:00:00Z'}}]}

Using the method of directly pinging the API

In [13]:
item_type = "PSScene"

# API request object
search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}

In [14]:
# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(API_KEY, ''),
    json=search_request)

# extract image IDs only
image_ids = [feature['id'] for feature in search_result.json()['features']]
print(len(image_ids))

250


Pagination links

In [15]:
search_result.json()['_links']

{'_first': 'https://api.planet.com/data/v1/searches/5bc94dbd94d046569788371b9fec3e89/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9wcmV2IjogZmFsc2UsICJxdWVyeV9wYXJhbXMiOiB7fX0%3D',
 '_next': 'https://api.planet.com/data/v1/searches/5bc94dbd94d046569788371b9fec3e89/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6ICIyMDIyLTEyLTIyVDIyOjQ1OjEzLjAwMDAwMFoiLCAic29ydF9sYXN0X2lkIjogIjIwMjIxMjIyXzE4MzEyNl8yM18yNDcwIiwgInNvcnRfcHJldiI6IGZhbHNlLCAicXVlcnlfcGFyYW1zIjoge319',
 '_self': 'https://api.planet.com/data/v1/searches/5bc94dbd94d046569788371b9fec3e89/results?_page=eyJwYWdlX3NpemUiOiAyNTAsICJzb3J0X2J5IjogInB1Ymxpc2hlZCIsICJzb3J0X2Rlc2MiOiB0cnVlLCAic29ydF9zdGFydCI6IG51bGwsICJzb3J0X2xhc3RfaWQiOiBudWxsLCAic29ydF9wcmV2IjogZmFsc2UsICJxdWVyeV9wYXJhbXMiOiB7fX0%3D'}

Now let's do it using the SDK

In [16]:
async with Session() as sess:
    cl = sess.client('data')
    item_list = [i async for i in cl.search(search_filter=combined_filter, item_types=item_types,limit=500)]

If the number of items requested is more than 250, the client will automatically fetch more pages of results in order to get the exact number requested.

Then we can save the output to be visualized as a geojson

In [17]:
len(item_list)

462

In [18]:
#Cloud cover filter
cloud_cover_filter = data_filter.range_filter('clear_percent', gt = 80)

#Combine all of the filters
combined_filter = data_filter.and_filter([geom_filter, date_range_filter, cloud_cover_filter])

async with Session() as sess:
    cl = sess.client('data')
    item_list = [i async for i in cl.search(search_filter=combined_filter, item_types=item_types,limit=500)]

In [19]:
len(item_list)

153

Now, we can iterate through our search results.

In [20]:
for item in item_list:
    print(item['id'], item['properties']['item_type'])

20230212_180808_26_24ca PSScene
20230129_180421_19_24bf PSScene
20230129_180418_82_24bf PSScene
20230206_180516_99_2415 PSScene
20230206_180514_61_2415 PSScene
20230206_180512_23_2415 PSScene
20230119_180752_40_24cc PSScene
20230129_180646_35_24c0 PSScene
20230129_180643_98_24c0 PSScene
20230220_180524_97_24af PSScene
20230215_180155_91_24b2 PSScene
20230219_183433_19_2488 PSScene
20230219_183437_75_2488 PSScene
20230219_183435_47_2488 PSScene
20230215_183614_29_2477 PSScene
20230215_183612_00_2477 PSScene
20230213_183645_48_2473 PSScene
20230213_183647_78_2473 PSScene
20230212_182219_33_2276 PSScene
20230212_182217_23_2276 PSScene
20230212_182215_13_2276 PSScene
20230212_184232_71_2424 PSScene
20230212_184228_28_2424 PSScene
20230212_184230_50_2424 PSScene
20230209_183619_29_2481 PSScene
20230209_183617_01_2481 PSScene
20230208_183511_46_248f PSScene
20230208_183509_15_248f PSScene
20230208_183513_77_248f PSScene
20230202_175730_69_241b PSScene
20230202_175735_10_241b PSScene
20230202

Here we can save all of our scene footprints as a geojson

In [21]:
scene_geoms = {
  "type": "FeatureCollection",
  "features": []
}

if not os.path.isdir('output'):
    os.mkdir('output')
else:
    if os.path.isfile('output/results.geojson'):
        os.remove('output/results.geojson')

with open('output/results.geojson','w') as f:
    for item in item_list:
        geom_out =     {
          "type": "Feature",
          "properties": item['properties'],
          "geometry": item['geometry']
        }
        scene_geoms['features'].append(geom_out)
    jsonStr = json.dumps(scene_geoms)
    f.write(jsonStr)
    f.close()

In [22]:
item_list[0]

{'_links': {'_self': 'https://api.planet.com/data/v1/item-types/PSScene/items/20230212_180808_26_24ca',
  'assets': 'https://api.planet.com/data/v1/item-types/PSScene/items/20230212_180808_26_24ca/assets/',
  'thumbnail': 'https://tiles.planet.com/data/v1/item-types/PSScene/items/20230212_180808_26_24ca/thumb'},
 '_permissions': ['assets.basic_analytic_4b:download',
  'assets.basic_analytic_4b_rpc:download',
  'assets.basic_analytic_4b_xml:download',
  'assets.basic_analytic_8b:download',
  'assets.basic_analytic_8b_xml:download',
  'assets.basic_udm2:download',
  'assets.ortho_analytic_4b:download',
  'assets.ortho_analytic_4b_sr:download',
  'assets.ortho_analytic_4b_xml:download',
  'assets.ortho_analytic_8b:download',
  'assets.ortho_analytic_8b_sr:download',
  'assets.ortho_analytic_8b_xml:download',
  'assets.ortho_udm2:download',
  'assets.ortho_visual:download'],
 'assets': ['basic_analytic_4b',
  'basic_analytic_4b_rpc',
  'basic_analytic_4b_xml',
  'basic_analytic_8b',
  'bas

## Ordering

Now that we have all of the imagery that we want to order we need to package it in a way that the Orders API can handle. Breaking it up by week.

In [23]:
grouped_items = []
current_group = []
#reverse the list since it comes in last date first
reversed_items = sorted(item_list, key=lambda item: item['properties']['acquired'])

#Select the earliest item
group_start_date = datetime.strptime(reversed_items[0]['properties']['acquired'], "%Y-%m-%dT%H:%M:%S.%fZ")

for item in reversed_items:
    time_object = item['properties']['acquired']
    time = datetime.strptime(time_object, "%Y-%m-%dT%H:%M:%S.%fZ")
    
    if time < group_start_date + timedelta(days=7):
        current_group.append(item)
        
    else:
        grouped_items.append(current_group)
        current_group = [item]
        group_start_date = time
if current_group:
    grouped_items.append(current_group)

print(len(grouped_items))

12


Lets see what the cloud cover is for our scenes

In [24]:
for item in grouped_items[0]:
    print(item['properties']['clear_percent'])

95
98
100
91
93
94
95
100


We need to sort the images to prioritize the clearer images ontop for the next step

In [25]:
sorted_items = []
for group in grouped_items:
    sorted_group = sorted(group, key=lambda item: item['properties']['clear_percent'], reverse=True)
    sorted_items.append(sorted_group)
    

for item in sorted_items[0]:
    print(item['properties']['clear_percent'])

100
100
98
95
95
94
93
91


In [26]:
def get_overlap(geometry1, geometry2):
    """Calculate the area of overlap between two geometries."""
    shape1 = shape(geometry1)
    shape2 = shape(geometry2)

    # Compute the intersection of the two geometries.
    intersection = shape1.intersection(shape2)

    return intersection

Look for the minimum about on scenes to cover the entire AOI

In [27]:
minimum_sorted_list = []


for week_items in sorted_items:
    intersection = False
    weekly_minimum_list = []
    for item in week_items:
        #for each scene itterate through every geometry and check if it overlaps with the scene
        overlap = get_overlap(geom_all, item['geometry'])
        if intersection:
            new_intersection = unary_union([overlap,intersection])

            #If the new interseciton is bigger then the old then add the scene to the order
            if round(new_intersection.area, 8) > round(intersection.area, 8):
                intersection = new_intersection
                weekly_minimum_list.append(item)
        else:
            if overlap.area > 0:
                intersection = overlap
                weekly_minimum_list.append(item)
    print(len(week_items), " to ", len(weekly_minimum_list))
    
    if len(weekly_minimum_list) > 0:
        minimum_sorted_list.append(weekly_minimum_list)

8  to  1
26  to  4
19  to  1
9  to  1
8  to  1
11  to  2
6  to  0
9  to  3
29  to  3
7  to  1
15  to  1
6  to  1


In [28]:
for item in sorted_items[0]:
    print(item['properties']['clear_percent'])
print("Now")
for item in minimum_sorted_list[0]:
    print(item['properties']['clear_percent'])

100
100
98
95
95
94
93
91
Now
98


Now lets print the average clear percent of each scene we are ordering

In [29]:
for group in minimum_sorted_list:
    clear = []
    for item in group:
        clear.append(int(item['properties']['clear_percent']))
    print(sum(clear)/len(clear))

98.0
98.5
98.0
93.0
91.0
95.0
92.33333333333333
97.66666666666667
98.0
100.0
97.0


We need to reverse the order of the scenes one more time because when mosaicing the last scene is stacked at the top

In [30]:
order_items = []
for group in minimum_sorted_list:
    sorted_group = sorted(group, key=lambda item: item['properties']['clear_percent'])
    order_items.append(sorted_group)
    
for item in order_items[0]:
    print(item['properties']['clear_percent'])

98


### Place a Order
Create the order structure using `planet` functions

In [ ]:
async def assemble_order(name,item_ids):
    products = [
        order_request.product(item_ids, 'analytic_udm2', 'PSScene')
    ]
    
    clip = order_request.clip_tool(aoi=geom_all)
    bandmath = order_request.band_math_tool(b1='(b2-b4)/(b2+b4)*100+100', pixel_type='8U')
    composite = order_request.composite_tool()

    
    
    tools = [clip,bandmath,composite]

    request = order_request.build_request(
        name, products=products, tools=tools)
    return request
    
request =  await assemble_order("test",['20230207_180504_51_24b6'])

In [ ]:
print(request)

Lets create a funcion to order imagery

In [ ]:
async def do_order(request):
    async with Session() as sess:
        cl = OrdersClient(sess)
        #with reporting.StateBar(state='creating') as bar:
        order = await cl.create_order(request)
        #bar.update(state='created', order_id=order['id'])

        await cl.wait(order['id'],max_attempts=0)#, callback=bar.update_state)
        os.mkdir(request['name'])
        
        # if we get here that means the order completed. Yay! Download the files.
        await cl.download_order(order['id'],directory=request['name'])


Now we can create all our orders

In [ ]:
order_list = []
folder_list= []
name = "lake_shasta_cloud_"
for group in order_items:
    ids = []
    order_name = name + group[0]['properties']['acquired'][:10]
    print(order_name)
    folder_list.append(order_name)
    for item in group:
        ids.append(item['id'])
    order_list.append(await assemble_order(order_name,ids))
print(len(order_list))

In [ ]:
nest_asyncio.apply()

#now all you need to do to have them run in parallel is to create an array of order requests
async with Session() as sess:
    tasks = [do_order(o) for o in order_list]
    await asyncio.gather(*tasks)

Now lets visualize our output!

In [ ]:
files = []
# for folder in folder_list:
#     files.extend(glob.glob(folder+"/*/composite.tif"))

files.extend(glob.glob("*/*/composite.tif"))
files.sort()
nrow = 4
ncol = 3

f, axes = plt.subplots(nrow, ncol, figsize=(3*ncol, 3*nrow))
for file, ax in zip(files, axes.flatten()):
    with rasterio.open(file) as src:
        arr = src.read()
    
    ax.imshow(arr[0], cmap="GnBu")
    
    
    date = file.split("_")[-1].split('/')[0]
    ax.set_title(date)

for ax in axes.flatten():
    ax.axis("off")
plt.tight_layout()

### Google Earth Engine Delivery

In order to deliver straight to earth engine you need to connect your gee account to a google cloud project and authorize planet to deliver to it. [Here](https://developers.planet.com/docs/integrations/gee/quickstart/) are some instrucitons to help with the process.

In [ ]:
async def assemble_order(item_ids, delivery=None):
    products = [
        order_request.product(item_ids, 'analytic_8b_sr_udm2', 'PSScene')
    ]
    tools = [order_request.clip_tool(aoi=geom_all)]
    request = order_request.build_request(
        'test_order_sdk', products=products, tools=tools, delivery=delivery)
    return request


delivery = order_request.google_earth_engine(
    project="planet-services-staging",
    collection="gis-day-gee-demo")


item_ids = [item["id"] for item in item_list]
request =  await assemble_order(item_ids, delivery=delivery)


# an async Orders client to request order creation
async with Session() as sess:
    cl = OrdersClient(sess)
    with reporting.StateBar(state='creating') as bar:
        # create order via Orders client
        order = await cl.create_order(request)
        bar.update(state='created', order_id=order['id'])
        await cl.wait(order['id'], callback=bar.update_state)